In [1]:
import numpy as np
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
import dill as pickle
import nltk

In [2]:
dataset=pd.read_csv('train_data.csv')
df=dataset[:]

In [3]:
df.head()

,Headline,Body ID,Stance,articleBody,Headline_processed,Body_processed
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\r\n...,"['polic', 'find', 'mass', 'grave', 'least', 'b...","['danni', 'boyl', 'direct', 'untitl', 'film', ..."
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...,"['hundr', 'palestinian', 'flee', 'flood', 'gaz...","['hundr', 'palestinian', 'evacu', 'home', 'sun..."
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...,"['christian', 'bale', 'pass', 'role', 'steve',...","['moscow', 'resid', 'hospit', 'wound', 'intim'..."
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...,"['hbo', 'appl', 'talk', 'appl', 'tv', 'stream'...","['reuter', 'canadian', 'soldier', 'shot', 'can..."
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's...","['spider', 'burrow', 'tourist', 'stomach', 'ch...","['fear', 'arachnophob', 'stori', 'bunburi', 's..."


In [4]:
def sentiment_process(df):
    print('generating sentiment features')
    print('for headline')
    
    sid = SentimentIntensityAnalyzer()
    
    def compute_sentiment(sentences):
        result = []
        for sentence in sentences:
            vs=sid.polarity_scores(sentence)
            result.append(vs)
        return pd.DataFrame(result).mean()
    
    df['Headline_sents']=df['Headline'].apply(lambda x: sent_tokenize(x))
    df = pd.concat([df, df['Headline_sents'].apply(lambda x: compute_sentiment(x))], axis=1)
    df.rename(columns={'compound':'h_compound', 'neg':'h_neg', 'neu':'h_neu', 'pos':'h_pos'}, inplace=True)
    headlineSenti = df[['h_compound','h_neg','h_neu','h_pos']].values
    print('headlineSenti.shape:')
    print(headlineSenti.shape)
    
    '''outfilename_hsenti_train = "headline.senti.pkl"
    with open(outfilename_hsenti_train, "wb") as outfile:
        pickle.dump(headlineSenti, outfile, -1)
    print('headline sentiment features of training set saved in ', outfilename_hsenti_train)'''
    
    print('for body')
    df['body_sents'] = df['articleBody'].map(lambda x: sent_tokenize(x))
    df = pd.concat([df, df['body_sents'].apply(lambda x: compute_sentiment(x))], axis=1)
    df.rename(columns={'compound':'b_compound', 'neg':'b_neg', 'neu':'b_neu', 'pos':'b_pos'}, inplace=True)
    bodySenti = df[['b_compound','b_neg','b_neu','b_pos']].values
    print('bodySenti.shape:')
    print(bodySenti.shape)
    
    '''outfilename_bsenti_train = "body.senti.pkl"
    with open(outfilename_bsenti_train, "wb") as outfile:
        pickle.dump(bodySenti, outfile, -1)
    print('body sentiment features of training set saved in ', outfilename_bsenti_train)'''
    
    return df

In [5]:
df_senti=sentiment_process(df)

generating sentiment features
for headline


D:\anaconda\installer\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


headlineSenti.shape:
(49972, 4)
for body
bodySenti.shape:
(49972, 4)


In [6]:
df_senti.to_csv('sentiment_features.csv')